# Imports were added on from old HW's we've had
* some aren't used

In [1]:
import pandas as pd
import tensorflow as tf

# Standard python helper libraries.
import os, sys, re, json, time
import itertools
import collections
from IPython.display import display

# NumPy and SciPy for matrix ops
import numpy as np
import scipy.sparse

# Pandas because pandas are awesome
import pandas as pd
# Set pandas floating point display
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# NLTK for NLP utils
import nltk

# Helper libraries
import utils
#import vocabulary

# Plotly imports.
import plotly.offline as plotly
plotly.offline.init_notebook_mode()
import plotly.graph_objs as go
from pprint import pprint
import sqlalchemy
import sqlite3

%matplotlib inline

from matplotlib import pyplot as plt
import time
import datetime

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


# Function I found just to list out table names

In [2]:
def importdb(db):
    
    conn = sqlite3.connect(db)
    c = conn.cursor()
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in c.fetchall():
        print (table)




In [3]:
importdb("reviews.db")

('reviews',)
('products',)
('categories',)


# Create pandas DF for reviews

In [4]:
conn = sqlite3.connect("reviews.db")
c = conn.cursor()
c.execute("Select * from reviews;")
review_table = pd.DataFrame(c.fetchall())
# Didn't know what the first column was

review_table.columns = ["??","rating","title","date","review","ASIN"]

# review_table.head()
review_table["rating"] = review_table.rating.apply(lambda x: int(x))
review_table["date"] = review_table.date.apply(lambda x: datetime.datetime.strptime(x, "%B %d, %Y"))
review_table.head()

,??,rating,title,date,review,ASIN
0,RL17959WYMDQO,5,for sizing,2015-07-23,I wear a size 8 in women's so I ordered a size...,B06XRW26VW
1,RPDE4MBK5YZ02,4,Seem to run a little narrow,2017-11-19,"Own several pair of these, this pair seems to ...",B06XRW26VW
2,R1T62C0ORV3J9E,5,My daughter is in love with these,2017-01-12,My daughter is in love with these. the first p...,B06XRW26VW
3,R1OA8AZV9I45U8,5,Just love these shoes,2017-09-18,Just love these shoes! And cheaper than anywhe...,B06XRW26VW
4,R21370ICYS03G6,5,"but like always, Amazon came through",2016-09-07,Met my expectations perfectly! They were sold ...,B06XRW26VW


In [5]:
review_table.shape

(62994, 6)

In [6]:
review_table.rating.value_counts()

5    43587
4    10357
3     4052
1     2831
2     2167
Name: rating, dtype: int64

In [7]:
len(np.unique(review_table.ASIN[review_table.rating<4]))

506

# doc_complete is what I use as the corpus that will be trained on
* below this is all reviews that have ratings less than 4

In [14]:
doc_complete = [i for i in review_table.review[review_table.rating<4]]

In [15]:
len(doc_complete)

6810

## Below I'm playing around with the idea of removing all words that only appear once as it probably wouldn't contribute to topic selection anyway. 

* May be a good idea to play around with tf_idf...I need to go through a refresher to gain familiarity with the subject

In [47]:
all_words = []

for doc in doc_complete:
    all_words.extend([x.lower() for x in doc.split(' ')])
from collections import Counter
counts = Counter(all_words)   

wc = pd.DataFrame.from_dict(counts,orient="index")

one_time = set(wc.index[wc[0]==1])

In [48]:
counts.most_common(100)

[('the', 10637),
 ('i', 7288),
 ('a', 5728),
 ('and', 5555),
 ('to', 4409),
 ('they', 3242),
 ('of', 3040),
 ('but', 2895),
 ('is', 2798),
 ('for', 2761),
 ('my', 2628),
 ('are', 2421),
 ('not', 2403),
 ('these', 2365),
 ('in', 2355),
 ('it', 2277),
 ('size', 1808),
 ('them', 1779),
 ('was', 1778),
 ('on', 1747),
 ('too', 1707),
 ('have', 1584),
 ('that', 1521),
 ('this', 1404),
 ('shoes', 1345),
 ('like', 1264),
 ('very', 1256),
 ('so', 1174),
 ('as', 1159),
 ('with', 1140),
 ('were', 1058),
 ('fit', 1044),
 ('be', 986),
 ('shoe', 958),
 ('had', 872),
 ('you', 866),
 ('just', 816),
 ('if', 805),
 ('adidas', 803),
 ('at', 767),
 ('would', 763),
 ('wear', 737),
 ('than', 720),
 ('after', 621),
 ('small', 613),
 ('one', 603),
 ('because', 596),
 ('ordered', 592),
 ('pair', 579),
 ('good', 579),
 ('or', 568),
 ('bought', 566),
 ('up', 549),
 ('when', 529),
 ("i'm", 528),
 ('way', 521),
 ('look', 519),
 ('will', 516),
 ('from', 515),
 ("don't", 509),
 ('little', 509),
 ('pants', 504),
 ('a

## doc clean below is the corpus cleansed of stop words, some punctuation and word stemming

In [31]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
#Extra stop words below
stop.update({'adidas','shoe','shoes','pant','pants'})
stop.update(one_time)
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete] 

In [32]:
import gensim

In [33]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
#row is document, first value is word, second is wordcount
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [34]:
# Just looking at format of dictionary...can ignore
for i in dictionary:
    if i < 10:
        print (i,dictionary[i])
    else:
        break
    

# ACTUAL MODEL

In [35]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=6, id2word = dictionary, passes=50)

## Playing around with what the package has to offer

In [40]:
ldamodel.show_topic(topicid=3)

[('foot', 0.027876126599764722),
 ('like', 0.014290692661209614),
 ('shoe', 0.014107389679758444),
 ('wear', 0.013363535699399251),
 ('would', 0.011863486618215579),
 ('really', 0.011563455880973936),
 ('comfortable', 0.011382413378347556),
 ('look', 0.010595276455863162),
 ('narrow', 0.010534889104093862),
 ('them', 0.0099304368665102411)]

In [37]:
ldamodel.print_topics(num_topics=6, num_words=2)

[(0, '0.027*"fit" + 0.021*"bag"'),
 (1, '0.021*"pair" + 0.017*"year"'),
 (2, '0.035*"color" + 0.019*"like"'),
 (3, '0.028*"foot" + 0.014*"like"'),
 (4, '0.023*"foot" + 0.022*"sandal"'),
 (5, '0.095*"size" + 0.032*"small"')]

# Ignore Below

In [ ]:
reviews = pd.read_csv("reviews.csv")

In [ ]:
twitter = pd.read_csv("twitter.csv",header=5)

In [ ]:
twitter.columns

In [ ]:
reviews[pd.isnull(reviews.Text)==False].shape

In [ ]:
reviews.head()

In [ ]:
import datetime

In [ ]:
datetime.datetime.strptime("03-JAN-17", '%d-%b-%y')

In [ ]:
def to_date(x):
    if pd.isnull(x) == False:
        return datetime.datetime.strptime(x, '%d-%b-%y')

In [ ]:
reviews["date"] = reviews["Submission Day"].apply(to_date)

In [ ]:
reviews["rating"] = reviews['Monthly Star Rating']

In [ ]:
reviews[['date','Product','Category','rating','Text']].head()

In [ ]:
import numpy as np
import re

## Later take a look at similarities by specific star rating
* Decided to look at full corpus for now

In [ ]:
one_star_text = ''

for text in reviews.Text[reviews.rating == 1]:
    one_star_text += '<s>' + re.sub(r'[^\w\s]',' ',re.sub("'",'',text.lower()))
    how_many += 1
    
    

In [ ]:
re.sub(' +',' ','The     quick brown    fox')

In [ ]:
twitter_text = ''

for text in twitter.Content:
    twitter_text += ' <s> ' + re.sub(r'[^\w\s]',' ',re.sub("'",'',str(text).lower()))

In [ ]:
twitter_text = re.sub(' +',' ',twitter_text)

In [ ]:
twitter_text[:300]

In [ ]:
amazon_text = ''
amazon_text_list = []

#how_many =0

for text in reviews.Text:
    amazon_text += ' <s> ' + re.sub(r'[^\w\s]',' ',re.sub("'",'',str(text).lower()))+ ' <\s> '
    #how_many+=1
    
for text in reviews.Text:
    amazon_text_list.append(' <s> ' + re.sub(r'[^\w\s]',' ',re.sub("'",'',str(text).lower()))+ ' <\s> ')   


In [ ]:
amazon_text = re.sub(' +',' ',all_text)
amazon_text[:500]

In [ ]:
amazon_words = amazon_text.split(" ")

In [ ]:
from collections import defaultdict

In [ ]:
word_count = defaultdict(int)



for word in amazon_words:
    word_count[word]+=1
    

    

In [ ]:
wc_df =pd.DataFrame.from_dict(word_count,orient="index")

In [ ]:
#wc_df['word'] = wc_df.index

In [ ]:
wc_df.reset_index(inplace=True)
wc_df.columns = ["word","count"]
wc_df['index_1'] = wc_df.index

In [ ]:
wc_df.sort_values(by=["count","word"],ascending=False,inplace=True)

In [ ]:
wc_df.tail()

In [ ]:
word_id = {}

for i,row in wc_df.iterrows():
    word_id[row.word]=i

In [ ]:
word_id['to']

In [ ]:
def cooccurrence_matrix(token_ids, V, K=2):
    # We'll use this as an "accumulator" matrix
    C = scipy.sparse.csc_matrix((V,V), dtype=np.float32)

    for k in range(1, K+1):
        print (u"Counting pairs (i, i \u00B1 %d) ..." % k)
        i = token_ids[:-k]  # current word
        j = token_ids[k:]   # k words ahead
        data = (np.ones_like(i), (i,j))  # values, indices
        Ck_plus = scipy.sparse.coo_matrix(data, shape=C.shape, dtype=np.float32)
        Ck_plus = scipy.sparse.csc_matrix(Ck_plus)
        Ck_minus = Ck_plus.T  # Consider k words behind
        C += Ck_plus + Ck_minus

    print ("Co-occurrence matrix: %d words x %d words" % (C.shape))
    print ("  %.02g nonzero elements" % (C.nnz))
    return C

In [ ]:
amazon_words[:10]

In [ ]:
# Show co-occurrence on a toy corpus


amazon_text_list

#words


amazon_words

# ?

# sentence_to_ids adds "<s>" and "</s>"

#gives id of words



amazon_words_ids = [word_id[s] for s in amazon_words]

# Here's the important part


amazon_C = cooccurrence_matrix(amazon_words_ids, len(amazon_words), K = 1)

#ORdered from most common to least common than reverse alphabetical

#pretty_print_matrix(amazon_C.toarray(), rows=wc_df.word, 
#                          cols=wc_df.word, dtype=int, highlight="> 0")

In [ ]:
def PPMI(C):
    """Tranform a counts matrix to PPMI.
    
    Args:
      C: scipy.sparse.csc_matrix of counts C_ij
    
    Returns:
      (scipy.sparse.csc_matrix) PPMI(C) as defined above
    """
    Z = float(C.sum())  # total counts
    # sum each column (along rows)
    Zc = np.array(C.sum(axis=0), dtype=np.float64).flatten()
    # sum each row (along columns)
    Zr = np.array(C.sum(axis=1), dtype=np.float64).flatten()
    
    # Get indices of relevant elements
    ii, jj = C.nonzero()  # row, column indices
    Cij = np.array(C[ii,jj], dtype=np.float64).flatten()
    
    ##
    # PMI equation
    pmi = np.log(Cij * Z / (Zr[ii] * Zc[jj]))
    ##
    # Truncate to positive only
    ppmi = np.maximum(0, pmi)  # take positive only
    
    # Re-format as sparse matrix
    ret = scipy.sparse.csc_matrix((ppmi, (ii,jj)), shape=C.shape,
                                  dtype=np.float64)
    ret.eliminate_zeros()  # remove zeros
    return ret

In [ ]:
from sklearn.decomposition import TruncatedSVD
def SVD(X, d=100):
    """Returns word vectors from SVD.
    
    Args:
      X: m x n matrix
      d: word vector dimension
      
    Returns:
      Wv : m x d matrix, each row is a word vector.
    """
    transformer = TruncatedSVD(n_components=d, random_state=1)
    Wv = transformer.fit_transform(X)
    # Normalize to unit length
    Wv = Wv / np.linalg.norm(Wv, axis=1).reshape([-1,1])
    return Wv, transformer.explained_variance_

In [ ]:
K = 1
d = 100
t0 = time.time()
C = cooccurrence_matrix(amazon_words_ids, len(amazon_words), K=K)

C_ppmi = PPMI(C)

Wv, _ = SVD(C_ppmi, d=d)


In [ ]:
n = 1000
data = [go.Scatter(x=Wv[:n,0], y=Wv[:n,1], text=amazon_words[:n],
                   mode='markers', textposition='bottom', hoverinfo='text')]
fig = go.Figure(data=data, layout=go.Layout(title="Word Embeddings", hovermode='closest'))
plotly.iplot(fig)